In [1]:
# Q-learning

import gym
import random
import numpy as np
import time
from gym.envs.registration import register   # we use this to get rid of the slippery stuff in frozen lake
from IPython.display import clear_output
from six import StringIO, b

In [2]:
#env_name = 'CartPole-v1'
#env_name = 'MountainCarContinuous-v0'
#env_name = 'Acrobot-v1'
#env_name = 'FrozenLake-v0'
#env_name = 'FrozenLakeNoSlip-v0'
#env_name = 'Taxi-v3'
#env_name = 'Gamble-v0'
#env_name = 'KellyCoinflip-v0'
env_name = 'Maze_edited-v0'

env = gym.make(env_name)
#goal_steps = 200

In [3]:
class Agent():
    def __init__(self, env):
        self.is_discrete = (type(env.action_space) == gym.spaces.discrete.Discrete)
        
        if self.is_discrete:
            self.action_size = env.action_space.n # how many actions are available - only works for discrete
            print("Action size: ", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
    
    def get_action(self, state): # choosing an action from the available actions
        if self.is_discrete:
            action = random.choice(range(self.action_size))  # discrete number of actions이기 때문에 이런 random 선정이 가능하다.
        else:
            action = np.random.uniform(self.action_low, self.action_high, self.action_shape)
        return action
    
agent = Agent(env)

Action size:  4


In [4]:
class QAgent(Agent):
    def __init__(self, env, discount_rate = 0.97, learning_rate = 0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n # 현 environment에서 가능한 discrete number of states를 값으로 받는다
        print("State size: ", self.state_size)    # in this example, 16 states exist (4*4 grid)
        
        self.eps = 0.1                            # exploration vs. exploitation. 1.0 means 100% random
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
    def build_model(self): # state의 갯수만큼 rows, action의 갯수만큼 column을 만들고, 랜덤하게 initializae한다.
        #self.q_table = 1e-4*np.random.random([self.state_size, self.action_size])
        self.q_table = np.zeros([self.state_size, self.action_size])
        for i in range(env.nrow):
            for j in range(env.ncol):
                if env.desc[i,j] in b'W':
                    for k in range(self.action_size):
                        self.q_table[i*env.nrow+j][k] = np.NINF

        
# #         second-best path
#         self.q_table[16][0] = -5
#         self.q_table[16][1] = -5
# #         self.q_table[16][2] = 4
#         self.q_table[16][3] = -5
        

#         self.q_table[83][0] = -5
#         self.q_table[83][1] = -5
# #         self.q_table[83][2] = 200
#         self.q_table[83][3] = -5
    
#         self.q_table[84][0] = -5
#         self.q_table[84][1] = -5
# #         self.q_table[83][2] = 200
#         self.q_table[84][3] = -5
    
#         self.q_table[85][0] = -5
#         self.q_table[85][1] = -5
#         self.q_table[85][2] = -5
        
#         self.q_table[94][0] = -5
#         self.q_table[94][1] = -5
#         self.q_table[94][2] = -5
        
#         self.q_table[103][0] = -5
#         self.q_table[103][1] = -5
# #         self.q_table[103][2] = 200
#         self.q_table[103][3] = -5
    
#         self.q_table[104][0] = -5
#         self.q_table[104][1] = -5
#         self.q_table[104][3] = -5
        
#         self.q_table[105][0] = -5
#         self.q_table[105][1] = -5
#         self.q_table[105][2] = -5
# #         self.q_table[105][3] = 200
        
        
        
        
        '''
        third-best path
        
        self.q_table[16][0] = -5
        self.q_table[16][1] = -5
#         self.q_table[16][2] = 4
        self.q_table[16][3] = -5
    
        self.q_table[56][0] = -5
        self.q_table[56][1] = -5
        self.q_table[56][3] = -5
#         self.q_table[56][3] = 10

        self.q_table[57][0] = -5
        self.q_table[57][1] = -5
        self.q_table[57][3] = -5
#         self.q_table[56][3] = 10

        self.q_table[58][0] = -5
        self.q_table[58][1] = -5
        self.q_table[58][3] = -5
#         self.q_table[56][3] = 10

        self.q_table[59][0] = -5
        self.q_table[59][1] = -5
        self.q_table[59][3] = -5
#         self.q_table[56][3] = 10

        self.q_table[60][0] = -5
        self.q_table[60][1] = -5
        self.q_table[60][3] = -5
#         self.q_table[56][3] = 10


        self.q_table[67][0] = -5
        self.q_table[67][1] = -5
        self.q_table[67][2] = -5
#         self.q_table[67][3] = 10
        
        self.q_table[78][0] = -5
        self.q_table[78][1] = -5
        self.q_table[78][2] = -5
#         self.q_table[78][3] = 10
        
#         self.q_table[79][0] = 10
        self.q_table[79][1] = -5
        self.q_table[79][2] = -5
        self.q_table[79][3] = -5
        
#         self.q_table[80][0] = 10
        self.q_table[80][1] = -5
        self.q_table[80][2] = -5
        self.q_table[80][3] = -5
        
#         self.q_table[81][0] = 10
        self.q_table[81][1] = -5
        self.q_table[81][2] = -5
        self.q_table[81][3] = -5
        
        self.q_table[92][0] = -5
        self.q_table[92][1] = -5
        self.q_table[92][2] = -5
#         self.q_table[92][3] = 10
        
        
        self.q_table[103][0] = -5
        self.q_table[103][1] = -5
        self.q_table[103][2] = -5
#         self.q_table[103][2] = 10
        
        '''
        
        
        
        
# #     optimal path

#         self.q_table[16][0] = -5
#         self.q_table[16][1] = -5
# #         self.q_table[16][2] = 4
#         self.q_table[16][3] = -5
    
# #         self.q_table[45][0] = -10
# #         self.q_table[45][1] = -10
# #         self.q_table[45][2] = -10
# # #         self.q_table[56][3] = 10
    
#         self.q_table[56][0] = -5
#         self.q_table[56][1] = -5
#         self.q_table[56][2] = -5
# #         self.q_table[56][3] = 10

#         self.q_table[67][0] = -5
#         self.q_table[67][1] = -5
#         self.q_table[67][2] = -5
# #         self.q_table[67][3] = 10
        
#         self.q_table[78][0] = -5
#         self.q_table[78][1] = -5
#         self.q_table[78][2] = -5
# #         self.q_table[78][3] = 10
        
# #         self.q_table[79][0] = 10
#         self.q_table[79][1] = -5
#         self.q_table[79][2] = -5
#         self.q_table[79][3] = -5
        
# #         self.q_table[80][0] = 10
#         self.q_table[80][1] = -5
#         self.q_table[80][2] = -5
#         self.q_table[80][3] = -5
        
# #         self.q_table[81][0] = 10
#         self.q_table[81][1] = -5
#         self.q_table[81][2] = -5
#         self.q_table[81][3] = -5
        
#         self.q_table[92][0] = -5
#         self.q_table[92][1] = -5
#         self.q_table[92][2] = -5
# #         self.q_table[92][3] = 10
        
        
#         self.q_table[103][0] = -5
#         self.q_table[103][1] = -5
#         self.q_table[103][2] = -5
# #         self.q_table[103][2] = 10

        

        '''
        LEFT = 0
        DOWN = 1
        RIGHT = 2
        UP = 3
        

        '''

    def get_action(self, state):
        '''
        Just going for the "best policy" was bad, because it completely depends on the initialization that we have made!
        Use randomness to search through the whole area, and as time passes we will reduce randomness
        
        learning rate 조정해서 해보기
        결과 정리
        논문
        '''
        q_state = self.q_table[state]                 # current state
        action_greedy = np.argmax(q_state)            # max q-value among the possible choices
        action_random = super().get_action(state)     # 상위 class인 agent의 get_action. 이건 random한거로 정의되어 있다
        if random.random() < self.eps:                # random하게 0~1 값과 eps 비교
            return action_random
        else:
            return action_greedy
        
    def get_action_greedy(self, state):
        q_state = self.q_table[state]                 # current state
        action_greedy = np.argmax(q_state)            # max q-value among the possible choices
        return action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = experience
        
        q_next = self.q_table[next_state]
        # current state is terminal
        if done:
            q_next = np.zeros([self.action_size])
        elif state == next_state:
            q_next = np.zeros([self.action_size])
        else:
            q_next
        
        # what the next action is based on the q-table
        q_target = reward + self.discount_rate*np.max(q_next)
        
        # update table
        q_update = q_target - self.q_table[state, action]             # 해당 행동이 table과 얼마나 차이 나는지 계산 (좋은 행동이면 양수)
        self.q_table[state, action] += self.learning_rate * q_update  # update q-table after applying learning rate
        
        # reduce randomness after each epoch
        
        # penalty 함수도 있어야한다.
        # reward heuristic 재구성! 가까워질수록 보상이 더 있는 것이 좋다.
agent = QAgent(env)

Action size:  4
State size:  121


In [5]:
print(env.observation_space)
print(agent.state_size, agent.action_size)
print(agent.eps)
env.render()
print(agent.q_table)
'''
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
벽으로 갈 때 강한 penalty 부여
'''

Discrete(121)
121 4
0.1

WWWWWWWWWWW
WPWPPPPPPGW
WPWPWPWWWWW
WPPPWPPPPPW
WPWWWWWWWPW
WPPPPPPWPPW
WPWWWWPWPWW
WPPPPWPPPPW
WWWWPWPWWWW
WSPPPPPPPPW
WWWWWWWWWWW
[[-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [  0.   0.   0.   0.]
 [-inf -inf -inf -inf]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [  0.   0.   0.   0.]
 [-inf -inf -inf -inf]
 [  0.   0.   0.   0.]
 [-inf -inf -inf -inf]
 [  0.   0.   0.   0.]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [-inf -inf -inf -inf]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0

'\nLEFT = 0\nDOWN = 1\nRIGHT = 2\nUP = 3\n벽으로 갈 때 강한 penalty 부여\n'

In [6]:
q_table_softmax = np.zeros([agent.state_size, agent.action_size])

# q-table softmax화
def softmax_array(array):
    for i in range(agent.state_size):
        q_table_softmax[i,:]  = np.exp(array[i,:])
        q_table_softmax[i,:] /= np.sum(q_table_softmax[i,:])
    return np.around(q_table_softmax, decimals = 2)
        
def print_softmax_array(array):
    for i in range(agent.state_size):
        q_table_softmax[i,:]  = np.exp(array[i,:])
        q_table_softmax[i,:] /= np.sum(q_table_softmax[i,:])
    q_table_softmax_final = np.around(q_table_softmax, decimals = 3)
    print(q_table_softmax_final)
        
print_softmax_array(agent.q_table)

[[ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [0.25 0.25 0.25 0.25]
 [ nan  nan  nan  nan]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [0.25 0.25 0.25 0.25]
 [ nan  nan  nan  nan]
 [0.25 0.25 0.25 0.25]
 [ nan  nan  nan  nan]
 [0.25 0.25 0.25 0.25]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [ nan  nan  nan  nan]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [ nan  nan

C:\Users\Jisoo\AppData\Local\Continuum\anaconda3\envs\gym\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


In [7]:
total_reward = 0
count_record = []
for i in range(3001):
    state = env.reset()
    done = False
    while not done:
    #    action = env.action_space.sample() # choosing a random action
        action = agent.get_action(state)                    # decide on an action
        next_state, reward, done, info = env.step(action)   # doing the random action in the environment. step은 이 4가지를 return 해준다.
#         print("state: ", state, "action: ", action)
#         print("Training Session: ", i, "    Total reward: ", total_reward, "    randomness: ", agent.eps)
        agent.train((state, action, next_state, reward, done))   # 해당 action에 대해 train한다
        state = next_state
        #env.render()                       # show the screen of the game
#         print(agent.q_table)               # show q-table after every action
        #time.sleep(0.2)                   # 약간의 딜레이 시간 추가
        clear_output(wait = True)         # 한 번에 1개의 action에 대한 q-table만 보이도록
    agent.eps = agent.eps * 0.99
    total_reward += reward
    

#     if i%100 == 0:
    state = env.reset()
    done = False
    count = 0
#     np.savetxt(r'D:\52Material\Lab\RL Experiment\Maze Environment\q_table%d.csv' % i, agent.q_table, delimiter=',')
#     np.savetxt(r'D:\52Material\Lab\RL Experiment\Maze Environment\softmax_q_table%d.csv' % i, softmax_array(agent.q_table), delimiter=',')
    while not done:
        action = agent.get_action_greedy(state)                    # decide on an action
        next_state, reward, done, info = env.step(action)   # doing the random action in the environment. step은 이 4가지를 return 해준다.
        count = count+1
#         print("state: ", state, "action: ", action)
#         print("Testing Session: ", i, "    Total reward: ", total_reward, "    randomness: ", agent.eps)
        if count>100:
            break
        else:
            state = next_state
#            env.render()                       # show the screen of the game
#             print(agent.q_table)               # show q-table after every action
#            time.sleep(0.4)                   # 약간의 딜레이 시간 추가
            clear_output(wait = True)         # 한 번에 1개의 action에 대한 q-table만 보이도록
#     print(count)
    count_record.append(count)


np.savetxt(r'D:\52Material\Lab\RL Experiment\Maze Environment\count_record.csv', count_record, delimiter=',')

env.close()


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
#agent.eps = 0
#agent.eps = 0.4

In [9]:
'''
12.9
reward에 대한 욕심이 없다? 그냥 계속 벽에 박는다. 앞으로 나아가는 것에 대해서 안좋게 인식.

Possible solutions:
중간에 reward
벽에 부딪치면 stronger punishment (개선 가능성이 적어보인다)
==>

'''

'''
벽에 부딪치는 것에 대한 punishment 증가 시킴.
randomness 0.2로 설정해서 하니까 잘찾는다.
100번으로는 학습이 잘 안된다.
==> 400번 하면 학습이 어느 정도 잘 된다.

더 먼 경로를 사전정보로 주면 어떤 결과가 있는지

아래에 길 하나 더 뚫어보기 (best, medium, worst path)


2000번 학습 시켜야지 최적 경로 정확하게 찾음...

20.01.21
initial 0.3, 0.99 try. 그리고 왔다갔다하는 구간 q-table 자세히 보기
gamble에서는 no prior 평균 q-table을 사전 정보로 줘서 학습 시켜보기 (같은 학습 방식으로)

20.01.12.
state 100 is where we begin
state 103에서 위로 가면 optimal, 우측으로 가면 2nd best (at best)
state 56에서 위로 가면 optimal, 우측으로 가면 3rd best (at best)

decision point에 대해서만 사전 정보를 주려면 엄청 크게 줘야지 학습 방향에 영향을 미친다.
3rd optimal을 생각한 것과 다르게 움직인다?


20.01.19
음수를 적용한 영향으로 사이 경로가 전체적으로 lower q-value를 가지게 된다. => 상충 시키게 영향의 +-를 합친 것을 0으로 해야하나?
+- 하니까 2개를 반복적으로 왔다갔다하는 문제가 있다.
'''

In [10]:
# for i in range(5):
#     state = env.reset()
#     done = False
#     while not done:
#     #    action = env.action_space.sample() # choosing a random action
#         action = agent.get_action(state)                    # decide on an action
#         next_state, reward, done, info = env.step(action)   # doing the random action in the environment. step은 이 4가지를 return 해준다.
#         print("state: ", state, "action: ", action)
#         print("Training Session: ", i, "    randomness: ", agent.eps)
#         agent.train((state, action, next_state, reward, done))   # 해당 action에 대해 train한다
#         state = next_state
#         env.render()                       # show the screen of the game
#         print(agent.q_table)               # show q-table after every action
#         time.sleep(0.05)                   # 약간의 딜레이 시간 추가
#         clear_output(wait = True)         # 한 번에 1개의 action에 대한 q-table만 보이도록
#     agent.eps = agent.eps * 0.95
# # reduce random faster

# # np.savetxt(r'D:\52Material\Lab\RL Experiment\Maze Environment\q_table.csv', agent.q_table, delimiter=',')
# # np.savetxt(r'D:\52Material\Lab\RL Experiment\Maze Environment\softmax_q_table.csv', softmax_array(agent.q_table), delimiter=',')

In [11]:
# for i in range(500):
#     state = env.reset()
#     done = False
#     while not done:
#     #    action = env.action_space.sample() # choosing a random action
#         action = agent.get_action(state)                    # decide on an action
#         next_state, reward, done, info = env.step(action)   # doing the random action in the environment. step은 이 4가지를 return 해준다.
# #         print("state: ", state, "action: ", action)
# #         print("Training Session: ", i, "    randomness: ", agent.eps)
#         agent.train((state, action, next_state, reward, done))   # 해당 action에 대해 train한다
#         state = next_state
# #         env.render()                       # show the screen of the game
# #         print(agent.q_table)               # show q-table after every action
# #         time.sleep(0.05)                   # 약간의 딜레이 시간 추가
#         clear_output(wait = True)         # 한 번에 1개의 action에 대한 q-table만 보이도록
#     agent.eps = agent.eps * 0.95
# # reduce random faster

# # np.savetxt(r'D:\52Material\Lab\RL Experiment\Maze Environment\q_table.csv', agent.q_table, delimiter=',')
# # np.savetxt(r'D:\52Material\Lab\RL Experiment\Maze Environment\softmax_q_table.csv', softmax_array(agent.q_table), delimiter=',')